# **Dependências do Projeto**

In [1]:
%pip install -U selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install torch torchvision torchaudio
#Instalando o pytorch

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
#Caso não consiga pydirectinput
%pip install pyautogui

Note: you may need to restart the kernel to use updated packages.


In [5]:
#Biblioteca de aprendizado por reforço
%pip install stable-baselines3[extra] protobuf==3.20.*

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install mss pydirectinput pytesseract

Note: you may need to restart the kernel to use updated packages.


# **Imports**

In [4]:
#Para captura de tela
from mss import mss
#import pyautogui #para comandos de teclado
import cv2 
#Tratamentos dos frames
import numpy as np 
import pyautogui
#Framework intermediário paara trabalhar com as imagens
#import pytesseract #para extrair o game over da imagem
from matplotlib import pyplot as plt #Visualizando resultados
import time
from PIL import Image
#Coisas para construir o ambinete
from gym import Env
from gym.spaces import Box, Discrete, Dict
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from io import BytesIO
from PIL import Image
from selenium.common.exceptions import WebDriverException
import base64
from io import BytesIO


In [5]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# **Construindo o Ambiente**

In [6]:
from collections import deque

In [7]:
#Classe para definir o ambiente
class WebGame(Env):
    #ESSAS FUNÇÃO SÃO NECESSÁRIAS POR CONTA DO GYM
    #Função responsavel pela inicialização do ambiente;
    #Portanto onde ficam todas as variaveis do ambiente, configuração das ações e da observação 
    def __init__(self):
        super().__init__()

        #Configuração do tempo
        self.print_time = .005
        self.wait_time = .015
        
        #Configuração do espaço
        #self.observation_space = Box(low=0, high=255, shape=(1,83,200), dtype=np.uint8)
        self.observation_space = Dict({
            "print":Box(low=0, high=255, shape=(1,83,200), dtype=np.uint8),
            "print2":Box(low=0, high=255, shape=(1,83,200), dtype=np.uint8),
            "print3":Box(low=0, high=255, shape=(1,83,200), dtype=np.uint8),
            #"print4":Box(low=0, high=255, shape=(1,83,200), dtype=np.uint8),
            "score":Box(low=0, high=4000, shape=(1,), dtype=np.uint8),
                                      })
        self.action_space = Discrete(2)

        #Variavei para a captura de tela
        self.screen_cap = mss() #Lib de captura de tela
        self.game_location = {"top": 300, "left": 0, "width": 650, "height": 500} #Área do monitor com 
        self.game_done = {"top": 40, "left": 0, "width": 800, "height": 640}

        self.chromedriver_path = "./chromedriver.exe"
        _chrome_options = webdriver.ChromeOptions()
        _chrome_options.add_argument("--mute-audio")
        _chrome_options.add_argument("--disable-gpu")
        self._driver = webdriver.Chrome(executable_path=self.chromedriver_path,options=_chrome_options)

        self.first_image = np.zeros(5)
        self.last_image = np.zeros((500,650,3))
        
        #Dicionário de ações
        self.action_dict = {
            0:Keys.ARROW_UP,
            1:'nothing'
        }

    #Função responsavel por passar as ações pro jogo, para fazer algo pro jogo
    def step(self, action):
        # Se ação escolhida for um dos botões, realizar a ação.
        if self.action_dict[int(action)] != 'nothing':
            self._driver.find_element(By.TAG_NAME, "body").send_keys(self.action_dict[int(action)])
        
        #Checa a próxima ação
        observation = self.get_observation()
        #Todas as ações tem que verificar se a ação acabou
        done, done_cap = self.get_done() 
        
        #Pegando o score do jogo
        score = self.get_score()
        #Ganhamos 1 ponto por cada frame que estamos vivos.
        reward = 1
        #É um dicionário de informações que retornam através do que a gente precisa
        info = {
            'score':score
        }
        
        return observation, reward, done, info
        

    #Função para a visualização do jogo:
    def render(self, mode: str='human'):
        img = cv2.cvtColor(self._get_image(), cv2.COLOR_BGR2RGB)
        if mode == 'rgb_array':
            return img
        elif mode == 'human':
            from gym.envs.classic_control import rendering
            if self.viewer is None:
                self.viewer = rendering.SimpleImageViewer()
            self.viewer.imshow(img)
            return self.viewer.isopen

        # cv2.imshow('Game', np.array(self.cap.grab(self.game_location))[:,:,:3])
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     self.close()

    #Função responsavel por recomeçar o jogo
    def reset(self):
        #Try catch precisa existir pq toda a vez q o selenium detecta que ele viu o chrome dino,
        #ele dispara um erro que esta sem internet.
        try:
            self._driver.get('chrome://dino')
            
        except WebDriverException:
            pass
        WebDriverWait(self._driver, 10).until(
            EC.presence_of_element_located((
                By.CLASS_NAME, 
                "runner-canvas"
            ))
        )
        self._driver.implicitly_wait(0.5)
        self._driver.find_elements(By.TAG_NAME, "body")[0].send_keys(Keys.ARROW_UP)
        

        return self.get_observation()

    #FUNÇÕES CUSTOM
    #Fecha a parte de visualização
    def close(self):
        cv2.destroyAllWindows()
    
    def get_score(self):
        score_array = self._driver.execute_script("return Runner.instance_.distanceMeter.digits")
        score = ''.join(score_array)
        #shape = np.zeros(1)
        #shape[0] = int(score)
        return int(score)
    
    def get_score_shape(self):
        shape = np.zeros(1)
        shape[0] = int(self.get_score())
        return shape
    
    def get_img(self):
        LEADING_TEXT = "data:image/png;base64,"
        canvas = self._driver.execute_script("return document.querySelector('canvas.runner-canvas').toDataURL()")
        img = canvas[len(LEADING_TEXT):]
        img_data = np.array(Image.open(BytesIO(base64.b64decode(img))))
        return img_data 
        # img = self._driver.execute_script()
    
    #Função responsavel por pegar a imagem do jogo e tratar da forma necessária.
    def get_print(self): 

        #img = numpy.array(self.screen_cap.grab(game_location))[:,:,3].astype(np.uint8)
        raw = np.array(self.get_img())[:,:,:3].astype(np.uint8)
        img = raw[:200,:400]
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (200,83))
        channel = np.reshape(resized, (1,83,200))
        
        self.first_image = self.last_image
        self.last_image = raw
        
        return channel
    
    #Função responsavel por pegar a imagem do jogo e tratar da forma necessária.
    def get_observation(self): 
        print1 = self.get_print()
        self._driver.implicitly_wait(self.print_time)
        print2 = self.get_print()
        self._driver.implicitly_wait(self.print_time)
        return {"print":print1, "print2":print2, "print3":self.get_print(), "score":self.get_score_shape() }
        #return self.get_print()


    #Função para pegar o texto de fim de jogo:
    def get_done(self):
        #Se as duas imagens iniciais forem iguais acabou jogo
        done = False
        #Colando a primeira condição devido ao fator de que as formas diferentes estavam dando problema na comparação devido ao np.all
        
        if self.first_image.shape != self.last_image.shape:
            return done, self.last_image

        if np.all(np.equal(self.first_image, self.last_image)) and not self._driver.execute_script("return Runner.instance_.playing"):
            done = True
        
        return done, self.last_image

# Treinamento

In [8]:
# Import os for file path management
import os 
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback, CheckpointCallback, CallbackList
# Verificando se o ambiente é válido para fazer as coisa
from stable_baselines3.common import env_checker

from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, SubprocVecEnv

In [9]:
from stable_baselines3 import DQN, PPO, DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, SubprocVecEnv, VecNormalize
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.logger import Figure, configure
from stable_baselines3.common.monitor import Monitor

In [10]:
def get_env():
    env = Monitor(WebGame())
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True,
                   clip_obs=10.)
    return env

In [11]:
#env_lambda = lambda: WebGame()
#env2 = SubprocVecEnv([env_lambda for i in range(4)])
env2 = get_env()

C:\Users\Dan\AppData\Local\Temp\ipykernel_4000\3477431199.py:33: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self._driver = webdriver.Chrome(executable_path=self.chromedriver_path,options=_chrome_options)


In [12]:
env2.reset()

OrderedDict([('print',
              array([[[[0., 0., 0., ..., 0., 0., 0.],
                       [0., 0., 0., ..., 0., 0., 0.],
                       [0., 0., 0., ..., 0., 0., 0.],
                       ...,
                       [0., 0., 0., ..., 0., 0., 0.],
                       [0., 0., 0., ..., 0., 0., 0.],
                       [0., 0., 0., ..., 0., 0., 0.]]]], dtype=float32)),
             ('print2',
              array([[[[0., 0., 0., ..., 0., 0., 0.],
                       [0., 0., 0., ..., 0., 0., 0.],
                       [0., 0., 0., ..., 0., 0., 0.],
                       ...,
                       [0., 0., 0., ..., 0., 0., 0.],
                       [0., 0., 0., ..., 0., 0., 0.],
                       [0., 0., 0., ..., 0., 0., 0.]]]], dtype=float32)),
             ('print3',
              array([[[[0., 0., 0., ..., 0., 0., 0.],
                       [0., 0., 0., ..., 0., 0., 0.],
                       [0., 0., 0., ..., 0., 0., 0.],
                       

In [13]:
#Criandos as pastas para que funcione.
CHECKPOINT_DIR = './checkpoints/'
LOG_DIR = './logs/'

In [19]:
checkpoint_callback = CheckpointCallback(
  save_freq=50000,
  save_path=CHECKPOINT_DIR,
  name_prefix="model",
  save_replay_buffer=True,
  save_vecnormalize=True,
    verbose=2
)
eval_callback = EvalCallback(env2, best_model_save_path=CHECKPOINT_DIR,
                             log_path=LOG_DIR, eval_freq=2000,
                             deterministic=True, render=False)
callback = CallbackList([checkpoint_callback, eval_callback])

In [20]:
def get_model():
    #model = PPO('MultiInputPolicy', env2, tensorboard_log=LOG_DIR, learning_rate=0.005, verbose=2, policy_kwargs=dict(normalize_images=False))
    model = DQN('MultiInputPolicy', env2, tensorboard_log=LOG_DIR, verbose=2, buffer_size=15000, 
        learning_starts=500, exploration_fraction=0.6, exploration_initial_eps=0.9, exploration_final_eps=0.05,
                policy_kwargs=dict(normalize_images=True))
    return model

In [21]:
model = get_model()

Using cpu device


E:\anaconda3\lib\site-packages\stable_baselines3\common\buffers.py:564: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 5.98GB > 4.40GB
  warnings.warn(


In [23]:
#Começa o treinamento
model._last_obs = None
model.learn(total_timesteps=250000, callback=callback, log_interval=10, reset_num_timesteps=False)

Logging to ./logs/DQN_0
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 125      |
|    ep_rew_mean      | 125      |
|    exploration_rate | 0.878    |
| time/               |          |
|    episodes         | 30       |
|    fps              | 17       |
|    time_elapsed     | 24       |
|    total_timesteps  | 3899     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00469  |
|    n_updates        | 849      |
----------------------------------
Eval num_timesteps=4000, episode_reward=241.60 +/- 50.21
Episode length: 241.60 +/- 50.21
----------------------------------
| eval/               |          |
|    mean_ep_length   | 242      |
|    mean_reward      | 242      |
| rollout/            |          |
|    exploration_rate | 0.878    |
| time/               |          |
|    total_timesteps  | 4000     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss  

Eval num_timesteps=16000, episode_reward=238.20 +/- 24.89
Episode length: 238.20 +/- 24.89
----------------------------------
| eval/               |          |
|    mean_ep_length   | 238      |
|    mean_reward      | 238      |
| rollout/            |          |
|    exploration_rate | 0.811    |
| time/               |          |
|    total_timesteps  | 16000    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00166  |
|    n_updates        | 3874     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 118      |
|    ep_rew_mean      | 118      |
|    exploration_rate | 0.81     |
| time/               |          |
|    episodes         | 130      |
|    fps              | 14       |
|    time_elapsed     | 889      |
|    total_timesteps  | 16102    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00182  |

Eval num_timesteps=28000, episode_reward=266.60 +/- 61.04
Episode length: 266.60 +/- 61.04
----------------------------------
| eval/               |          |
|    mean_ep_length   | 267      |
|    mean_reward      | 267      |
| rollout/            |          |
|    exploration_rate | 0.744    |
| time/               |          |
|    total_timesteps  | 28000    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000916 |
|    n_updates        | 6874     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 119      |
|    ep_rew_mean      | 119      |
|    exploration_rate | 0.741    |
| time/               |          |
|    episodes         | 230      |
|    fps              | 14       |
|    time_elapsed     | 1735     |
|    total_timesteps  | 28470    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000972 |

Eval num_timesteps=40000, episode_reward=222.00 +/- 14.01
Episode length: 222.00 +/- 14.01
----------------------------------
| eval/               |          |
|    mean_ep_length   | 222      |
|    mean_reward      | 222      |
| rollout/            |          |
|    exploration_rate | 0.676    |
| time/               |          |
|    total_timesteps  | 40000    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000704 |
|    n_updates        | 9874     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 114      |
|    ep_rew_mean      | 114      |
|    exploration_rate | 0.676    |
| time/               |          |
|    episodes         | 330      |
|    fps              | 14       |
|    time_elapsed     | 2528     |
|    total_timesteps  | 40122    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00272  |

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 115      |
|    ep_rew_mean      | 115      |
|    exploration_rate | 0.616    |
| time/               |          |
|    episodes         | 420      |
|    fps              | 14       |
|    time_elapsed     | 3292     |
|    total_timesteps  | 50807    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00121  |
|    n_updates        | 12576    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 114      |
|    ep_rew_mean      | 114      |
|    exploration_rate | 0.61     |
| time/               |          |
|    episodes         | 430      |
|    fps              | 14       |
|    time_elapsed     | 3355     |
|    total_timesteps  | 51823    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00214  |
|    n_updates      

...

Saving model checkpoint to ./checkpoints/model_200000_steps.zip
Saving model replay buffer checkpoint to ./checkpoints/model_replay_buffer_200000_steps.pkl
Saving model VecNormalize to ./checkpoints/model_vecnormalize_200000_steps.pkl
Eval num_timesteps=200000, episode_reward=252.00 +/- 59.33
Episode length: 252.00 +/- 59.33
----------------------------------
| eval/               |          |
|    mean_ep_length   | 252      |
|    mean_reward      | 252      |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 200000   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000787 |
|    n_updates        | 49874    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 120      |
|    ep_rew_mean      | 120      |
|    exploration_rate | 0.05     |
| time/               |          |
|    epi

Eval num_timesteps=212000, episode_reward=389.20 +/- 183.27
Episode length: 389.20 +/- 183.27
----------------------------------
| eval/               |          |
|    mean_ep_length   | 389      |
|    mean_reward      | 389      |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 212000   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00195  |
|    n_updates        | 52874    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 126      |
|    ep_rew_mean      | 126      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1830     |
|    fps              | 13       |
|    time_elapsed     | 15619    |
|    total_timesteps  | 212254   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00409

Eval num_timesteps=224000, episode_reward=209.40 +/- 26.07
Episode length: 209.40 +/- 26.07
----------------------------------
| eval/               |          |
|    mean_ep_length   | 209      |
|    mean_reward      | 209      |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 224000   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000562 |
|    n_updates        | 55874    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 125      |
|    ep_rew_mean      | 125      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1930     |
|    fps              | 13       |
|    time_elapsed     | 16501    |
|    total_timesteps  | 225113   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00234  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 127      |
|    ep_rew_mean      | 127      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 2020     |
|    fps              | 13       |
|    time_elapsed     | 17323    |
|    total_timesteps  | 236962   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000221 |
|    n_updates        | 59115    |
----------------------------------
Eval num_timesteps=238000, episode_reward=308.60 +/- 35.10
Episode length: 308.60 +/- 35.10
----------------------------------
| eval/               |          |
|    mean_ep_length   | 309      |
|    mean_reward      | 309      |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 238000   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0039   

Saving model checkpoint to ./checkpoints/model_250000_steps.zip
Saving model replay buffer checkpoint to ./checkpoints/model_replay_buffer_250000_steps.pkl
Saving model VecNormalize to ./checkpoints/model_vecnormalize_250000_steps.pkl
Eval num_timesteps=250000, episode_reward=166.00 +/- 2.10
Episode length: 166.00 +/- 2.10
----------------------------------
| eval/               |          |
|    mean_ep_length   | 166      |
|    mean_reward      | 166      |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 250000   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000904 |
|    n_updates        | 62374    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 136      |
|    ep_rew_mean      | 136      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episo

# Teste

In [25]:
env2 = get_env()
model = get_model()
model.load('checkpoints/model_200000_steps') 

for episode in range(5): 
    obs = env2.reset()
    done = False
    total_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env2.step(action)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))

    

C:\Users\Dan\AppData\Local\Temp\ipykernel_4000\3477431199.py:33: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self._driver = webdriver.Chrome(executable_path=self.chromedriver_path,options=_chrome_options)


Using cpu device
Total Reward for episode 0 is [29.24366]
Total Reward for episode 1 is [8.525146]
Total Reward for episode 2 is [10.116716]
Total Reward for episode 3 is [8.206295]
Total Reward for episode 4 is [7.6212187]


In [24]:
%tensorboard --logdir ./logs/DQN_0/ --host localhost

In [ ]:
CREATE A SECTION FOR RESUMING TESTS